In [2]:
import json
import numpy as np
from IPython.display import display, HTML
import pandas as pd
pd.set_option("display.precision", 3)

In [3]:
file_path = "../result.json"

result_data = []

# Open the file and load JSON data
with open(file_path, 'r') as json_file:
    for line in json_file:
        try:
            result_data.append(json.loads(line))
        except:
            continue

In [4]:
planner_names = list(set([d['planner_name'] for d in result_data]))

planner_names = [
    "MTGTaskPlanner",
    "MTGTaskPlannerWithGMM",
    # "DynamicMTGTaskPlannerWithGMM_25.0",
    # "DynamicMTGTaskPlannerWithGMM_50.0",
    # "DynamicMTGTaskPlannerWithGMM_75.0",
    # "DynamicMTGTaskPlannerWithGMM_100.0",
    # "DynamicMTGTaskPlannerWithGMM_125.0",
#     "DynamicMTGTaskPlannerWithGMM_150.0",
    "MTGTaskPlannerWithAtlas",
    # "DynamicMTGTaskPlannerWithAtlas_25.0",
    # "DynamicMTGTaskPlannerWithAtlas_50.0",
    # "DynamicMTGTaskPlannerWithAtlas_75.0",
    # "DynamicMTGTaskPlannerWithAtlas_100.0",
    # "DynamicMTGTaskPlannerWithAtlas_125.0",
#     "DynamicMTGTaskPlannerWithAtlas_150.0",
]

metrics = [
    'motion_planning_time',
    'task_planning_time',
    'updating_time',
    'solution_length',
    'num_attempts',
]

planner_stats = {
                    planner_name: {
                        'num' : 0, 
                        'num_success' : [], 
                        'motion_planning_time' : [],
                        'task_planning_time' : [],
                        'updating_time' : [],
                        'solution_length' : [],
                        'num_attempts' : [],
                        'total_planning_time' : [],
                    } 
                 
                 for planner_name in planner_names
                }

for d in result_data:
    if d['planner_name'] in planner_stats:
        dic = planner_stats[d['planner_name']]
        dic['num'] += 1
        dic['num_success'].append(d['success'] == "true")
        if d['success'] == "true":
            dic['motion_planning_time'].append(d['motion_planning_time'])
            dic['task_planning_time'].append(d['task_planning_time'])
            dic['total_planning_time'].append(d['total_planning_time'])
            dic['updating_time'].append(d['updating_time'])
            dic['solution_length'].append(d['solution_length'])
            dic['num_attempts'].append(d['num_attempts'])

# print(
#         "{0: >6}".format("Planner Name"), 
#         "{0: >6}".format("Success Rate"),
#         "{0: >6}".format("Mean Task Planning Time"),
#         "{0: >6}".format("Mean Motion Planning Time"),
#         "{0: >6}".format("Mean Total Planning Time"),
#         "{0: >6}".format("Mean Update time"),
#         "{0: >6}".format("Mean solution length"),
#         "{0: >6}".format("Mean number of attempts"),
#     )        

output_dict = []
for planner_name in planner_names:
    stats = planner_stats[planner_name]

    # print(
    #         "{0: >37}".format(planner_name), 
    #         "{:.3f}%".format(np.mean(stats['num_success']) * 100.0) , 
    #         "{:.3f}".format(np.mean(np.array(stats['task_planning_time']) / (np.array(stats['num_attempts'])) )), 
    #         "{:.3f}".format(np.mean(np.array(stats['motion_planning_time']) / (np.array(stats['num_attempts'])) )), 
    #         "{:.3f}".format(np.mean(np.array(stats['total_planning_time']) / (np.array(stats['num_attempts'])) )),
    #         "{:.3f}".format(np.mean(np.array(stats['updating_time']) / (np.array(stats['num_attempts'])) )), 
    #         "{:.3f}".format(np.mean(np.array(stats['solution_length']) / (np.array(stats['num_attempts'])) )), 
    #         "{:.3f}".format(np.mean((np.array(stats['num_attempts'])) )),

    #     )
    current_planner_eval_dict = {}
    current_planner_eval_dict["Planner Name"] = planner_name
    current_planner_eval_dict["Success Rate"] = np.mean(stats['num_success']) * 100.0
    current_planner_eval_dict["Mean Task Planning Time"] = np.mean(np.array(stats['task_planning_time']) / (np.array(stats['num_attempts'])) )
    current_planner_eval_dict["Mean Motion Planning Time"] = np.mean(np.array(stats['motion_planning_time']) / (np.array(stats['num_attempts'])) )
    current_planner_eval_dict["Mean Total Planning Time"] = np.mean(np.array(stats['total_planning_time']) )
    current_planner_eval_dict["Mean Update time"] = np.mean(np.array(stats['updating_time']) / (np.array(stats['num_attempts'])) )
    current_planner_eval_dict["Mean solution length"] = np.mean(np.array(stats['solution_length']) / (np.array(stats['num_attempts'])) )
    current_planner_eval_dict["Mean number of attempts"] = np.mean((np.array(stats['num_attempts'])) )
    output_dict.append(current_planner_eval_dict)
df = pd.DataFrame(output_dict).set_index('Planner Name')

display(df)

,Mean Motion Planning Time,Mean Task Planning Time,Mean Total Planning Time,Mean Update time,Mean number of attempts,Mean solution length,Success Rate
Planner Name,,,,,,,
MTGTaskPlanner,2.806,0.028,5.605,0.062,1.667,12.970,60.0
MTGTaskPlannerWithGMM,2.540,0.085,4.396,0.506,1.333,10.779,60.0
MTGTaskPlannerWithAtlas,2.457,0.084,4.163,0.531,1.333,10.211,60.0


In [5]:
import pandas as pd

df = pd.DataFrame(result_data)
df

,goal,motion_planning_time,num_attempts,planner_name,solution_length,start,success,task_planning_time,total_planning_time,updating_time
0,"[0, 0]",2.472,1,MTGTaskPlanner,15.758,"[0, 8]",true,0.016,2.489,8.106e-06
1,"[0, 0]",11.481,3,MTGTaskPlanner,35.070,"[0, 9]",true,0.148,12.190,5.600e-01
2,"[0, 11]",-1.000,-1,MTGTaskPlanner,-1.000,"[0, 5]",false,-1.000,-1.000,-1.000e+00
3,"[0, 5]",-1.000,-1,MTGTaskPlanner,-1.000,"[0, 2]",false,-1.000,-1.000,-1.000e+00
4,"[0, 8]",2.118,1,MTGTaskPlanner,11.464,"[0, 0]",true,0.018,2.137,8.821e-06
5,"[0, 0]",2.163,1,MTGTaskPlannerWithGMM,11.632,"[0, 8]",true,0.066,2.765,5.349e-01
6,"[0, 0]",2.239,1,MTGTaskPlannerWithGMM,12.637,"[0, 9]",true,0.065,2.836,5.307e-01
7,"[0, 11]",-1.000,-1,MTGTaskPlannerWithGMM,-1.000,"[0, 5]",false,-1.000,-1.000,-1.000e+00
8,"[0, 5]",-1.000,-1,MTGTaskPlannerWithGMM,-1.000,"[0, 2]",false,-1.000,-1.000,-1.000e+00
9,"[0, 8]",6.437,2,MTGTaskPlannerWithGMM,16.136,"[0, 0]",true,0.245,7.588,9.051e-01


In [6]:
for groupname, group in df.groupby('planner_name', sort=False):
    print(groupname)
    display(group.describe().loc[['mean', 'std', '50%', 'max']])

MTGTaskPlanner


,motion_planning_time,num_attempts,solution_length,task_planning_time,total_planning_time,updating_time
mean,2.814,0.600,12.058,-0.363,2.963,-2.880e-01
std,5.119,1.673,14.871,0.584,5.419,6.890e-01
50%,2.118,1.000,11.464,0.016,2.137,8.106e-06
max,11.481,3.000,35.070,0.148,12.190,5.600e-01


MTGTaskPlannerWithGMM


,motion_planning_time,num_attempts,solution_length,task_planning_time,total_planning_time,updating_time
mean,1.768,0.400,7.681,-0.325,2.238,-0.006
std,3.062,1.342,8.099,0.621,3.543,0.920
50%,2.163,1.000,11.632,0.065,2.765,0.531
max,6.437,2.000,16.136,0.245,7.588,0.905


MTGTaskPlannerWithAtlas


,motion_planning_time,num_attempts,solution_length,task_planning_time,total_planning_time,updating_time
mean,1.619,0.400,7.252,-0.334,2.098,0.013
std,2.710,1.342,7.693,0.608,3.182,0.938
50%,2.240,1.000,11.129,0.065,2.866,0.559
max,5.447,2.000,15.261,0.148,6.541,0.944
